# Evaluate Model Outputs

<div class="alert alert-block alert-danger">
<b> Project Name Setting: </b> Please set the Project Name for your project </div>

In [ ]:
# CHANGE PROJECT NAME HERE
PROJECT_NAME = 'Blandy'

# CURRENT DIRECTORY HERE, SET WHERE THE NOTEBOOKS ARE LOCATED
CURRENT_DIRECTORY = ''

# SET THE SPATIAL RESOLUTION (in meters)
SPATIAL_RESOLUTION = 30

In [ ]:
import os, shutil
os.chdir(CURRENT_DIRECTORY)

In [ ]:
# Define folder paths as global variables
PROJECT_DIR = os.path.join(CURRENT_DIRECTORY, PROJECT_NAME)
RAWGIS_DIR = os.path.join(PROJECT_DIR, "gis_data")
RAWOBS_DIR = os.path.join(PROJECT_DIR, "obs")
RAWSOIL_DIR = os.path.join(RAWGIS_DIR, "soil")
MODEL_DIR = os.path.join(PROJECT_DIR, 'model')
DEF_DIR = os.path.join(MODEL_DIR, 'defs')
INI_DIR = os.path.join(MODEL_DIR, 'ini_files')
EPC_DIR = os.path.join(MODEL_DIR, 'epc_files')
OUTPUT_DIR = os.path.join(MODEL_DIR, 'output')
CO2_DIR = os.path.join(MODEL_DIR, 'co2')
NDEP_DIR = os.path.join(MODEL_DIR, 'ndep')
ENDPOINT_DIR = os.path.join(MODEL_DIR, 'endpoint_files')
SPINUP_DIR = os.path.join(MODEL_DIR, 'spinup')
NORMAL_DIR = os.path.join(MODEL_DIR, 'normal')
MODEL_RAST_DIR = os.path.join(MODEL_DIR, 'raster_inputs')
IMAGE = os.path.join(PROJECT_DIR, 'image_map')

In [ ]:
import xarray as xr
import scipy
import pandas as pd
import re
from datetime import datetime, timedelta, date
import numpy as np
import matplotlib.pyplot as plt
from csv import reader
import math
import pyproj
from pyproj import CRS, Transformer, Proj, transform
import dask.array as da
import multiprocessing
from time import time
import glob
from matplotlib.ticker import FormatStrFormatter

### Read in Model Outputs for Each Site

In [ ]:
# opened_final = xr.open_mfdataset(os.path.join(OUTPUT_DIR, "*_n.nc"))
blandy = xr.open_mfdataset(os.path.join(CURRENT_DIRECTORY, 'Blandy', 'model', 'output', "*.nc"))
serc = xr.open_mfdataset(os.path.join(CURRENT_DIRECTORY, 'SERC', 'model', 'output', "*.nc"))
coweeta = xr.open_mfdataset(os.path.join(CURRENT_DIRECTORY, 'Coweeta', 'model', 'output', "*.nc"))

blandy

### Read Ameriflux Data

In [ ]:
# Read in observed data from Ameriflux towers - data retrieved from https://ameriflux.lbl.gov
blandy_flux = pd.read_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', 'AMF_US-xBL_BASE_HH_5-5.csv'), skiprows=[0,1]) # Dataset is now stored in a Pandas Dataframe
coweeta_flux = pd.read_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', 'AMF_US-Cwt_BASE_HR_1-5.csv'), skiprows=[0,1]) # Dataset is now stored in a Pandas Dataframe
serc_flux = pd.read_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', 'AMF_US-xSE_BASE_HH_6-5.csv'), skiprows=[0,1]) # Dataset is now stored in a Pandas Dataframe

In [ ]:
#Index Flux data with datetime

flux_dataset_list = {'blandy': blandy_flux, 'coweeta': coweeta_flux, 'serc': serc_flux}

for name, df_flux in flux_dataset_list.items():
    DateTime_column = 'TIMESTAMP_START'

    df_flux[DateTime_column] = df_flux[DateTime_column].astype(int).astype(str)
    df_flux.index= pd.to_datetime(df_flux[DateTime_column], format="%Y%m%d%H%M")
    
    if name != 'coweeta':
        df_flux_short = df_flux[['LE', 'NEE_PI']].copy().replace(-9999,np.NaN)
    else:
        df_flux_short = df_flux[['LE', 'NEE_PI_F']].copy().replace(-9999,np.NaN)

    # resample each value column separately, and count the number of NaN values in each day
#     resampled_value1 = df_flux_short['P'].resample('D')
    resampled_value2 = df_flux_short['LE'].resample('D')
    if name != 'coweeta':
        resampled_value3 = df_flux_short['NEE_PI'].resample('D')
    else:
        resampled_value3 = df_flux_short['NEE_PI_F'].resample('D')
#     resampled_value3 = df_flux_short['VPD_PI'].resample('D')
#     resampled_value4 = df_flux_short['SWC_1_2_1'].resample('D')

    # combine the resampled data into a single dataframe
    resampled = pd.concat([resampled_value2.mean(), resampled_value3.mean()], axis=1)
    daily_count_nan = pd.concat([resampled_value2.apply(lambda x: x.isna().sum()), resampled_value3.apply(lambda x: x.isna().sum())], axis=1)
    
    resampled['ET'] = (resampled['LE']*60*60*24)/ 2257 / 1000
    daily_count_nan['ET'] = daily_count_nan['LE'] 
    
    resampled.to_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', f'{name}.csv'))
    daily_count_nan.to_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', f'{name}_num_missing.csv'))
    

### Create Summary Plot

In [ ]:
# CREATE PLOT TEMPLATE
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(16,16))

output_model = {'blandy': blandy, 'coweeta': coweeta, 'serc': serc}

ax1 = 0
for name, opened_final in output_model.items():
    
    ax = axs[0,ax1]
        
    annual_nee = opened_final["daily_nee"].mean(dim='time').rename("NEE (kg C/m^2/year)")
    # daily_gpp = opened_final["daily_gpp"].isel(time=[135,136,137,138])
    annual_nee_plot = annual_nee.plot.imshow(x="long", y="lat", robust=True, cmap='YlGn', add_colorbar=False, vmin=-0.00008, vmax=0.00008, ax=ax)
    # nee.fig.suptitle('Annual Cumulative GPP', fontsize='x-large')
    
    if ax1 != 0:
        ax.set_xticklabels([])
        ax.set_xlabel('')
        ax.set_yticklabels([])
        ax.set_ylabel('')

    tower_data = pd.read_csv(os.path.join(CURRENT_DIRECTORY, 'Ameriflux', f'{name}.csv'))
    tower_data.set_index('TIMESTAMP_START', inplace=True)
    tower_data.index = pd.to_datetime(tower_data.index)
    
    if name == 'coweeta':
        tower_data['NEE_PI'] = tower_data['NEE_PI_F']
        start_date = '2011-01-01'
        end_date = '2015-12-31'
    else:
        start_date = '2017-01-01'
        end_date = '2022-06-30'
    
    tower_data["NEE_PI"] = tower_data["NEE_PI"]*(0.001037664)

    obs_nee = tower_data["NEE_PI"]

    # FILTER OUTLIER DATA FROM OBSERVED
    q_low = obs_nee.quantile(0.025)
    q_hi  = obs_nee.quantile(0.975)
    obs_filtered = obs_nee[(obs_nee < q_hi) & (obs_nee > q_low)]
    
    ax = axs[1,ax1]
    
    # COMBINE OBSERVED AND MODELLED VALUES AND FILTER BY TIME RANGE OF INTEREST, PLOT COMPARITIVE TIME SERIES
    blandy_nee_mean = opened_final["daily_nee"].median(dim='lat', skipna=True)
    blandy_nee_mean = blandy_nee_mean.mean(dim='long', skipna=True)
    blandy_nee_mean = blandy_nee_mean.sel(time=slice(start_date, end_date))
    blandy_nee_mean_series = blandy_nee_mean.to_series()
    blandy_nee_mean_series.index

    nee_combined = pd.DataFrame({'Modelled NEE': blandy_nee_mean_series,'Observed NEE': obs_filtered})
    mask = (nee_combined.index > start_date) & (nee_combined.index <= end_date)
    nee_combined = nee_combined.loc[mask]
    ax.set_ylim([-0.009, 0.009])
    
    if ax1 != 0:
        nee_combined.plot(xlabel='Time', ylabel='NEE (kgC/m^2/day)', ax=ax, legend=False)
        ax.set_yticklabels([])
        ax.set_ylabel('')
    else:
        nee_combined.plot(xlabel='Time', ylabel='NEE (kgC/m^2/day)', ax=ax)

#     # CALCULATE PEARSONS R CORRELATION COEFFICIENT
#     pearson_r_grid = nee_combined.corr(method='pearson')
#     pearson_r = pearson_r_grid.at['Observed NEE', 'Modelled NEE']
#     print('Pearson r: ', round(pearson_r, 2))

    # MANUAL METHOD FOR CALCULATING PEARSON COEFFICIENT R
    # n = nee_combined["Observed NEE"].count()
    # sum_xy = (nee_combined["Observed NEE"]*nee_combined["Modelled NEE"]).sum()
    # nee_combined["Modelled_removed_na"] = nee_combined["Modelled NEE"] + nee_combined["Observed NEE"] - nee_combined["Observed NEE"]

    # r = ((n)*sum_xy - (nee_combined["Modelled_removed_na"].sum()*nee_combined["Observed NEE"].sum()))/((((n*(nee_combined["Modelled_removed_na"]**2).sum())-(nee_combined["Modelled_removed_na"].sum())**2)*((n*(nee_combined["Observed NEE"]**2).sum())-(nee_combined["Observed NEE"].sum())**2))**(1/2))
    # print(round(r,2))

#     #CALCULATE NSE
#     nee_combined['nse_top'] = (nee_combined["Observed NEE"] - nee_combined["Modelled NEE"])**2
#     nee_combined['nse_bottom'] = (nee_combined["Observed NEE"] - (nee_combined["Observed NEE"]).mean())**2
#     NSE = 1 - (nee_combined['nse_top'].sum() / nee_combined['nse_bottom'].sum())
#     print(f'NSE: {round(NSE, 2)}')

#     # CALCULATE KGE: https://agrimetsoft.com/calculators/Kling-Gupta%20efficiency
#     nee_combined["Modelled_removed_na"] = nee_combined["Modelled NEE"] + nee_combined["Observed NEE"] - nee_combined["Observed NEE"]
#     KGE = 1 - ((pearson_r-1)**2+((nee_combined["Modelled_removed_na"].std()/nee_combined["Observed NEE"].std())-1)**2+((nee_combined["Modelled_removed_na"].mean()/nee_combined["Observed NEE"].mean())-1)**2)**(1/2)
#     print('KGE: ', round(KGE, 2),  '\n')

#     plt.show()
#     print('\n')

#     # ------------------------------------------------------------------------------------------------------------------------------------------------------------------
    #PLOT RESIDUALS
    ax = axs[2,ax1]
    
    nee_combined['residual'] = nee_combined["Modelled NEE"] - nee_combined["Observed NEE"]

    ax.scatter(nee_combined["Modelled NEE"], nee_combined['residual'], color='black')
    ax.axhline(y=0, color='k', linestyle='--')
    
    ax.set_xlim([-0.01, 0.01])
    ax.set_ylim([-0.01, 0.01])
    
    if ax1 != 0:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
    else:
        ax.set_xlabel('Observed NEE')
        ax.set_ylabel('Residual of Predicted NEE')
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
        

    ax1 += 1

axs[0,0].text(0.02, 0.88, "A", transform=axs[0,0].transAxes, size=20, weight='bold')
axs[0,1].text(0.02, 0.88, "B", transform=axs[0,1].transAxes, size=20, weight='bold')
axs[0,2].text(0.02, 0.88, "C", transform=axs[0,2].transAxes, size=20, weight='bold')
axs[1,0].text(0.02, 0.88, "D", transform=axs[1,0].transAxes, size=20, weight='bold')
axs[1,1].text(0.02, 0.88, "E", transform=axs[1,1].transAxes, size=20, weight='bold')
axs[1,2].text(0.02, 0.88, "F", transform=axs[1,2].transAxes, size=20, weight='bold')
axs[2,0].text(0.02, 0.88, "G", transform=axs[2,0].transAxes, size=20, weight='bold')
axs[2,1].text(0.02, 0.88, "H", transform=axs[2,1].transAxes, size=20, weight='bold')
axs[2,2].text(0.02, 0.88, "I", transform=axs[2,2].transAxes, size=20, weight='bold')

axs[0,0].set_title('Blandy',size=16)
axs[0,1].set_title('Coweeta', size=16)
axs[0,2].set_title('SERC', size=16)

fig.subplots_adjust(wspace=0.05)

cbar = fig.colorbar(annual_nee_plot, ax=axs[0, :], shrink=0.25, location="top", pad=0.15)
cbar.ax.set_title('NEE (kg C/m^2/yr)')

plt.show()

### Some other Plots

In [ ]:
annual_nee = blandy["daily_nee"].mean(dim='time').rename("NEE (kg C/m^2/year)")

# daily_gpp = opened_final["daily_gpp"].isel(time=[135,136,137,138])
nee = annual_nee.plot.imshow(x="long", y="lat", robust=True, figsize=(8, 8), cmap='YlGn', add_colorbar=False, vmin=-0.00008, vmax=0.00008)
# nee.fig.suptitle('Annual Cumulative GPP', fontsize='x-large')

plt.show()

In [ ]:
gpp_agg = opened_final["daily_gpp"]
gpp_agg = gpp_agg.resample(time="Y").sum().mean(dim='lat', skipna=True).mean(dim='long', skipna=True)
gpp_agg.to_series().plot(title='Annual Average GPP over AOI', xlabel='Time', ylabel='GPP (kg C / m^2 / yr)', color='forestgreen')

plt.show()

In [ ]:
nee_agg = opened_final["daily_nee"]
nee_agg = nee_agg.resample(time="Y").sum().mean(dim='lat', skipna=True).mean(dim='long', skipna=True)
nee_agg.to_series().plot(title='Annual Average NEE over AOI', xlabel='Time', ylabel='NEE (kg C / m^2 / yr)', color='forestgreen')
plt.axhline(y=0.0, color='black', linestyle='dotted')

plt.show()

In [ ]:
et_agg = opened_final["evapotransp"]
et_agg = et_agg.resample(time="Y").sum().mean(dim='lat', skipna=True).mean(dim='long', skipna=True).rename("ET")
# et_agg['time'] = et_agg['time'].astype(int)
et_agg.to_series().plot(title='Annual ET and P over AOI', xlabel='Time', ylabel='ET (mm)', legend=True, color='lightsteelblue')

precip = met_data["prcp"].resample(time="Y").sum().mean(dim='y', skipna=True).mean(dim='x', skipna=True).rename("Precipitation")
ax = precip.to_series().plot(secondary_y=False, legend=True)

ax.set_ylabel('Depth (mm)')

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

runoff_snk = opened_final["runoff_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
runoff_snk['Lag'] = runoff_snk["runoff_snk"].shift(1).fillna(0)
runoff_snk['runoff'] = runoff_snk["runoff_snk"] - runoff_snk['Lag']

soilevap_snk = opened_final["soilevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
soilevap_snk['Lag'] = soilevap_snk["soilevap_snk"].shift(1).fillna(0)
soilevap_snk['soilevap'] = soilevap_snk["soilevap_snk"] - soilevap_snk['Lag']

canopyevap_snk = opened_final["canopyevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
canopyevap_snk['Lag'] = canopyevap_snk["canopyevap_snk"].shift(1).fillna(0)
canopyevap_snk['canopyevap'] = canopyevap_snk["canopyevap_snk"] - canopyevap_snk['Lag']

pondwevap_snk = opened_final["pondwevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
pondwevap_snk['Lag'] = pondwevap_snk["pondwevap_snk"].shift(1).fillna(0)
pondwevap_snk['pondwaterevap'] = pondwevap_snk["pondwevap_snk"] - pondwevap_snk['Lag']

trans_snk = opened_final["trans_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
trans_snk['Lag'] = trans_snk["trans_snk"].shift(1).fillna(0)
trans_snk['trans'] = trans_snk["trans_snk"] - trans_snk['Lag']

groundwater_src = opened_final["groundwater_src"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
groundwater_src['Lag'] = groundwater_src["groundwater_src"].shift(1).fillna(0)
groundwater_src['groundwater'] = groundwater_src["groundwater_src"] - groundwater_src['Lag']

deeppercolation_snk = opened_final["deeppercolation_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
deeppercolation_snk['Lag'] = deeppercolation_snk["deeppercolation_snk"].shift(1).fillna(0)
deeppercolation_snk['deeppercolation'] = deeppercolation_snk["deeppercolation_snk"] - deeppercolation_snk['Lag']

water_budget = pd.concat([runoff_snk['runoff'], soilevap_snk['soilevap'], canopyevap_snk['canopyevap'], pondwevap_snk['pondwaterevap'], trans_snk['trans'], deeppercolation_snk['deeppercolation'], groundwater_src['groundwater']], axis=1)
water_budget = water_budget.resample("Y").sum()

precip = met_data["prcp"].resample(time="Y").sum().mean(dim='y', skipna=True).mean(dim='x', skipna=True)
precip_frame = precip.to_series().to_frame()

water_budget = pd.concat([water_budget, precip_frame], axis=1)
water_budget.index = water_budget.index.year

fig = plt.figure()
ax = water_budget[["runoff", "soilevap", "canopyevap", "pondwaterevap", "trans", "deeppercolation", "groundwater"]].plot(kind='bar', stacked=True, title='Annual Water Budget', color=['lightsteelblue','darkgoldenrod','forestgreen','midnightblue','mediumseagreen','bisque', 'Blue'])
ax.plot(water_budget[['prcp']].values, linestyle='-', linewidth=2.0)
ax.set_xlabel('Time')
ax.set_ylabel('Depth (mm)')
# plt.legend(["Precipitation", "ET", "Runoff", "Deep Leaching/Percolation"]) 
# combined_hydro[['prcp']].plot(kind='line', color='black', ms=10)

plt.show()

# cumulative_frame['daily'].resample("Y").sum().plot()
# plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True

runoff_snk = opened_final["runoff_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
runoff_snk['Lag'] = runoff_snk["runoff_snk"].shift(1).fillna(0)
runoff_snk['runoff'] = runoff_snk["runoff_snk"] - runoff_snk['Lag']

soilevap_snk = opened_final["soilevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
soilevap_snk['Lag'] = soilevap_snk["soilevap_snk"].shift(1).fillna(0)
soilevap_snk['soilevap'] = soilevap_snk["soilevap_snk"] - soilevap_snk['Lag']

canopyevap_snk = opened_final["canopyevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
canopyevap_snk['Lag'] = canopyevap_snk["canopyevap_snk"].shift(1).fillna(0)
canopyevap_snk['canopyevap'] = canopyevap_snk["canopyevap_snk"] - canopyevap_snk['Lag']

pondwevap_snk = opened_final["pondwevap_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
pondwevap_snk['Lag'] = pondwevap_snk["pondwevap_snk"].shift(1).fillna(0)
pondwevap_snk['pondwaterevap'] = pondwevap_snk["pondwevap_snk"] - pondwevap_snk['Lag']

trans_snk = opened_final["trans_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
trans_snk['Lag'] = trans_snk["trans_snk"].shift(1).fillna(0)
trans_snk['trans'] = trans_snk["trans_snk"] - trans_snk['Lag']

deeppercolation_snk = opened_final["deeppercolation_snk"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
deeppercolation_snk['Lag'] = deeppercolation_snk["deeppercolation_snk"].shift(1).fillna(0)
deeppercolation_snk['deeppercolation'] = deeppercolation_snk["deeppercolation_snk"] - deeppercolation_snk['Lag']

soilw_SUM = opened_final["soilw_SUM"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
soilw_SUM['Lag'] = soilw_SUM["soilw_SUM"].shift(1).fillna(0)
soilw_SUM['soilw'] = soilw_SUM["soilw_SUM"] - soilw_SUM['Lag']

prcp_src = opened_final["prcp_src"].mean(dim='lat', skipna=True).mean(dim='long', skipna=True).to_series().to_frame()
prcp_src['Lag'] = prcp_src["prcp_src"].shift(1).fillna(0)
prcp_src['prcp'] = prcp_src["prcp_src"] - prcp_src['Lag']

water_budget = pd.concat([runoff_snk['runoff'], soilevap_snk['soilevap'], canopyevap_snk['canopyevap'], pondwevap_snk['pondwaterevap'], trans_snk['trans'], deeppercolation_snk['deeppercolation'], groundwater_src['groundwater'], soilw_SUM['soilw'], prcp_src['prcp']], axis=1)
water_budget.index = water_budget.index.date
water_budget = water_budget.iloc[425:440]

fig = plt.figure()
ax = water_budget[["soilw", "deeppercolation", "soilevap", "runoff", "pondwaterevap", "canopyevap", "trans"]].plot(kind='bar', stacked=True, title='Daily Water Budget', color=['saddlebrown', 'bisque', 'darkgoldenrod','lightsteelblue','midnightblue','forestgreen','mediumseagreen'])
ax.plot(water_budget[['prcp']].values, linestyle='-', linewidth=2.0)
ax.set_xlabel('Time')
ax.set_ylabel('Depth (mm)')
plt.legend(["Precipitation", "Soil Water", "Deep Percolation", "Soil Evap", "Runoff", "Pond Water Evap", "Canopy Evap", "Transpiration"], bbox_to_anchor=(1, 1)) 
# ax.legend(bbox_to_anchor=(1, 1))

plt.show()

### Map Sample of Outputs

In [ ]:
# %matplotlib inline

In [ ]:
annual_gpp = opened_final["daily_gpp"].resample(time="Y", skipna=False).sum(skipna=False).isel(time=[37,38,39,40]).rename("GPP (kg C/m^2/year)")
# daily_gpp = opened_final["daily_gpp"].isel(time=[135,136,137,138])
gpp = annual_gpp.plot.imshow(x="long", y="lat", col="time", col_wrap=2, robust=True, figsize=(8, 8), cmap='YlGn', cbar_kwargs={'location': 'right', 'anchor': (2.5, 0.5), 'panchor': (1.0, 0.5)})
gpp.fig.suptitle('Annual Cumulative GPP', fontsize='x-large')

start_year = 2017
for i, ax in enumerate(gpp.axes.flatten()):
    ax.set_title('Year: %d' % (start_year + i))

plt.show()

In [ ]:
annual_nee = opened_final["daily_nee"].resample(time="Y", skipna=False).sum(skipna=False).isel(time=[37,38,39,40]).rename("NEE (kg C/m^2/year)")
nee = annual_nee.plot.imshow(x="long", y="lat", col="time", col_wrap=2, robust=True, figsize=(8, 8), cmap='summer', cbar_kwargs={'location': 'right', 'anchor': (2.5, 0.5), 'panchor': (1.0, 0.5)})
nee.fig.suptitle('Annual Cumulative NEE', fontsize='x-large')

start_year = 2017
for i, ax in enumerate(nee.axes.flatten()):
    ax.set_title('Year: %d' % (start_year + i))

plt.show()

In [ ]:
evapotransp = opened_final["evapotransp"].resample(time="Y", skipna=False).sum(skipna=False).isel(time=[37,38,39,40]).rename("ET (mm/year)")
et = evapotransp.plot.imshow(x="long", y="lat", col="time", col_wrap=2, robust=True, figsize=(8, 8), cmap='Blues', cbar_kwargs={'location': 'right', 'anchor': (2.5, 0.5), 'panchor': (1.0, 0.5)})
et.fig.suptitle('Annual Cumulative ET', fontsize='x-large')

start_year = 2017
for i, ax in enumerate(et.axes.flatten()):
    ax.set_title('Year: %d' % (start_year + i))

plt.show()

In [ ]:
lai = opened_final["proj_lai"].resample(time="Y", skipna=False).max(skipna=False).isel(time=[37,38,39,40]).rename("Max Annual LAI")
lai = lai.plot.imshow(x="long", y="lat", col="time", col_wrap=2, robust=True, figsize=(8, 8), cmap='YlGn', cbar_kwargs={'location': 'right', 'anchor': (2.5, 0.5), 'panchor': (1.0, 0.5)})
lai.fig.suptitle('Max Annual LAI', fontsize='x-large')

start_year = 2017
for i, ax in enumerate(lai.axes.flatten()):
    ax.set_title('Year: %d' % (start_year + i))

plt.show()